In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On Simple English wiki, POS tagged

sf = open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/enwiki_20200520_10pc_tagged.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

Autism|NN is|VBZ a|DT developmental|JJ disorder|NN characterized|VBN by|IN difficulties|NNS with|IN social|JJ interaction|NN and|CC communication|NN and|CC by|IN restricted|JJ and|CC repetitive|JJ behavior|NN 

Parents|NNS often|RB notice|VBP signs|NNS during|IN the|DT first|JJ three|CD years|NNS of|IN their|PRP$ child|NN 's|POS life|NN 

These|DT signs|NNS often|RB develop|VBP gradually|RB though|IN some|DT children|NNS with|IN autism|NN experience|NN worsening|VBG in|IN their|PRP$ communication|NN and|CC social|JJ skills|NNS after|IN reaching|VBG developmental|JJ milestones|NNS at|IN a|DT normal|JJ pace|NN 

Autism|NN is|VBZ associated|VBN with|IN a|DT combination|NN of|IN genetic|JJ and|CC environmental|JJ factors|NNS 

Risk|NN factors|NNS during|IN pregnancy|NN include|VBP certain|JJ infections|NNS such|JJ as|IN rubella|NN toxins|NNS including|VBG valproic|JJ acid|NN alcohol|NN cocaine|NN pesticides|NNS lead|VBP and|CC air|NN pollution|NN fetal|JJ growth|NN restriction|NN and|CC au

In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

from nltk.corpus.reader.util import read_line_block

In [5]:
w10p_t = PlaintextCorpusReader(datapath+'/Corpora/wiki/enwiki_20200520/Tagged','enwiki_20200520_10pc_tagged.txt',
                            word_tokenizer = WhitespaceTokenizer(),
                            para_block_reader=read_line_block
                            )

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [7]:
from nltk.util import ngrams, everygrams

In [8]:

i = 0
for sent in word_gen(w10p_t, remchars = "[`¬@^\"]"):
    i += 1
    print(sent)
    if i > 20 :break

Autism|NN
is|VBZ
a|DT
developmental|JJ
disorder|NN
characterized|VBN
by|IN
difficulties|NNS
with|IN
social|JJ
interaction|NN
and|CC
communication|NN
and|CC
by|IN
restricted|JJ
and|CC
repetitive|JJ
behavior|NN
|^|
Parents|NNS


In [9]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [10]:
scorer = NgramAssocMeasures.poisson_stirling

In [11]:
min_freq = 20

In [12]:
tri_cf = TrigramCollocationFinder.from_words(word_gen(w10p_t,remchars = "[`¬@^\"]"))
tri_cf.apply_freq_filter(min_freq)
tri_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [13]:
bi_cf = tri_cf.bigram_finder()
bi_cf.apply_freq_filter(min_freq)
bi_cf.apply_word_filter(lambda w: w in ('|^|'))  # Filter out associations with sentence boundary marker

In [14]:
print("\t", [" ".join(tup) for tup in bi_cf.nbest(scorer, 15)])

	 ['311.22|CD 500.86|CD', 'Dechawat|NNP Poomjaeng|NNP', 'MSC1|NN MSC2|NN', 'Makhaya|NNP Ntini|NNP', 'resting_place_coordinates|NNS burial_place|VBP', 'Deshin|NNP Shekpa|NNP', 'Neophron|NNP percnopterus|NNP', 'Eugeniya|NNP Pashkova|NNP', 'Pirke|NNP De-Rabbi|NNP', 'ISTFEA|NNP ISTSPR|NNP', 'tikkun|NNP olam|NNP', 'Librarie|NNP Sansaisha|NNP', 'burial_place|VBP burial_coordinates|NNS', 'Frecce|NNP Tricolori|NNP', 'Tsvetana|NNP Pironkova|NNP']


In [15]:
bi_dict = {}
tri_dict = {}

In [16]:
for bigram in bi_cf.score_ngrams(scorer):
    fq = bi_cf.ngram_fd[bigram[0]]
    if fq >= min_freq:
        bi_dict[bigram[0]] = [bigram[0], fq, bigram[-1]]
    
bi_dict

{('311.22|CD', '500.86|CD'): [('311.22|CD', '500.86|CD'),
  20,
  -667.0842188911779],
 ('Dechawat|NNP', 'Poomjaeng|NNP'): [('Dechawat|NNP', 'Poomjaeng|NNP'),
  20,
  -667.0842188911779],
 ('MSC1|NN', 'MSC2|NN'): [('MSC1|NN', 'MSC2|NN'), 20, -667.0842188911779],
 ('Makhaya|NNP', 'Ntini|NNP'): [('Makhaya|NNP', 'Ntini|NNP'),
  20,
  -667.0842188911779],
 ('resting_place_coordinates|NNS',
  'burial_place|VBP'): [('resting_place_coordinates|NNS',
   'burial_place|VBP'), 20, -667.0842188911779],
 ('Deshin|NNP', 'Shekpa|NNP'): [('Deshin|NNP', 'Shekpa|NNP'),
  20,
  -668.4920054490058],
 ('Neophron|NNP', 'percnopterus|NNP'): [('Neophron|NNP', 'percnopterus|NNP'),
  20,
  -668.4920054490058],
 ('Eugeniya|NNP', 'Pashkova|NNP'): [('Eugeniya|NNP', 'Pashkova|NNP'),
  20,
  -669.8342893661766],
 ('Pirke|NNP', 'De-Rabbi|NNP'): [('Pirke|NNP', 'De-Rabbi|NNP'),
  20,
  -669.8342893661766],
 ('ISTFEA|NNP', 'ISTSPR|NNP'): [('ISTFEA|NNP', 'ISTSPR|NNP'),
  20,
  -669.8997920068338],
 ('tikkun|NNP', 'olam|N

In [17]:
for trigram in tri_cf.score_ngrams(scorer):
    fq = tri_cf.ngram_fd[trigram[0]]
    if fq >= min_freq:
        tri_dict[trigram[0]] = [trigram[0], fq, trigram[-1]]
    
tri_dict

{('resting_place_coordinates|NNS',
  'burial_place|VBP',
  'burial_coordinates|NNS'): [('resting_place_coordinates|NNS',
   'burial_place|VBP',
   'burial_coordinates|NNS'),
  20,
  -757.5554580123182],
 ('resting_place|NN',
  'resting_place_coordinates|NNS',
  'burial_place|VBP'): [('resting_place|NN',
   'resting_place_coordinates|NNS',
   'burial_place|VBP'), 20, -761.0930132539997],
 ('body_discovered|VBN',
  'resting_place|NN',
  'resting_place_coordinates|NNS'): [('body_discovered|VBN',
   'resting_place|NN',
   'resting_place_coordinates|NNS'),
  20,
  -767.5315751517469],
 ('CiconiiformesFamily|NNP',
  'Ciconiidae|NNP',
  'Storks|NNPS'): [('CiconiiformesFamily|NNP',
   'Ciconiidae|NNP',
   'Storks|NNPS'), 20, -784.1042067855074],
 ('RBRFEA|NNP',
  'RBRSPR|NNP',
  'SILFEA|NNP'): [('RBRFEA|NNP',
   'RBRSPR|NNP',
   'SILFEA|NNP'), 20, -789.24216012447],
 ('Rolie|NNP',
  'Polie|NNP',
  'Olie|NNP'): [('Rolie|NNP',
   'Polie|NNP',
   'Olie|NNP'), 20, -790.6346029431342],
 ('Çimento|N

In [18]:
bigram_df = pd.DataFrame.from_dict(bi_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

del bi_dict

bigram_df

,ngram,freq,poisson
"(311.22|CD, 500.86|CD)","(311.22|CD, 500.86|CD)",20,-6.670842e+02
"(Dechawat|NNP, Poomjaeng|NNP)","(Dechawat|NNP, Poomjaeng|NNP)",20,-6.670842e+02
"(MSC1|NN, MSC2|NN)","(MSC1|NN, MSC2|NN)",20,-6.670842e+02
"(Makhaya|NNP, Ntini|NNP)","(Makhaya|NNP, Ntini|NNP)",20,-6.670842e+02
"(resting_place_coordinates|NNS, burial_place|VBP)","(resting_place_coordinates|NNS, burial_place|VBP)",20,-6.670842e+02
...,...,...,...
"(on|IN, the|DT)","(on|IN, the|DT)",473024,-2.576335e+07
"(and|CC, the|DT)","(and|CC, the|DT)",492576,-2.778679e+07
"(to|IN, the|DT)","(to|IN, the|DT)",714100,-3.884106e+07
"(in|IN, the|DT)","(in|IN, the|DT)",1388181,-7.588717e+07


In [19]:
trigram_df = pd.DataFrame.from_dict(tri_dict, orient='index',
                       columns=['ngram', 'freq', 'poisson'])

del tri_dict

In [20]:
ngram_df = bigram_df.append(trigram_df).sort_values('poisson', ascending=False).reset_index(drop=True)

In [21]:
ngram_df['len'] = ngram_df.ngram.apply(len)

In [22]:
ngram_df

,ngram,freq,poisson,len
0,"(311.22|CD, 500.86|CD)",20,-6.670842e+02,2
1,"(Makhaya|NNP, Ntini|NNP)",20,-6.670842e+02,2
2,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-6.670842e+02,2
3,"(Dechawat|NNP, Poomjaeng|NNP)",20,-6.670842e+02,2
4,"(MSC1|NN, MSC2|NN)",20,-6.670842e+02,2
...,...,...,...,...
1859185,"(on|IN, the|DT)",473024,-2.576335e+07,2
1859186,"(and|CC, the|DT)",492576,-2.778679e+07,2
1859187,"(to|IN, the|DT)",714100,-3.884106e+07,2
1859188,"(in|IN, the|DT)",1388181,-7.588717e+07,2


In [23]:
del bigram_df, trigram_df

In [24]:
# Pickle ngram_df - can re-use later with different approaches (ngram_df = pd.read_pickle() )
ngram_df.to_pickle(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/ngram_df.pkl')

In [25]:
# ngram_df = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_df.pkl')

In [26]:
len(w10p_t.words())

261006802

In [27]:
# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(word_gen(w10p_t, remchars = "[`¬@^\"]", sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [28]:
stop

{'%|NN',
 '0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'but|CC',
 'by|IN',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'its|PRP$',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 's|POS',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'the|DT',
 'to|IN',
 'to|TO',
 'two|CD',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'who|WP',
 'with|IN',
 '|HYPH',
 '–|SYM'}

In [29]:
import pickle

In [30]:
with open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/stop.pkl', 'wb') as pfile:
    pickle.dump(stop,pfile)

In [31]:
from batcher import batcher  # Custom module with logic for assigning n-grams to batches, avoiding overlap

In [32]:
# Duplicate entries appearing for some reason. Removing here
ngram_df2 = ngram_df[ngram_df.freq >= min_freq].drop_duplicates().reset_index(drop=True)

ngram_df2

,ngram,freq,poisson,len
0,"(311.22|CD, 500.86|CD)",20,-6.670842e+02,2
1,"(Makhaya|NNP, Ntini|NNP)",20,-6.670842e+02,2
2,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-6.670842e+02,2
3,"(Dechawat|NNP, Poomjaeng|NNP)",20,-6.670842e+02,2
4,"(MSC1|NN, MSC2|NN)",20,-6.670842e+02,2
...,...,...,...,...
1859185,"(on|IN, the|DT)",473024,-2.576335e+07,2
1859186,"(and|CC, the|DT)",492576,-2.778679e+07,2
1859187,"(to|IN, the|DT)",714100,-3.884106e+07,2
1859188,"(in|IN, the|DT)",1388181,-7.588717e+07,2


In [33]:
eval_count = 500000

ngram_eval = ngram_df2[0:eval_count]

ngram_eval

,ngram,freq,poisson,len
0,"(311.22|CD, 500.86|CD)",20,-667.084219,2
1,"(Makhaya|NNP, Ntini|NNP)",20,-667.084219,2
2,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-667.084219,2
3,"(Dechawat|NNP, Poomjaeng|NNP)",20,-667.084219,2
4,"(MSC1|NN, MSC2|NN)",20,-667.084219,2
...,...,...,...,...
499995,"(US|NNP, Steel|NNP)",32,-1679.033919,2
499996,"(Harry|NNP, Wilson|NNP)",32,-1679.034986,2
499997,"(a|DT, solo|JJ, piano|NN)",23,-1679.035464,3
499998,"(1962|CD, album|NN)",30,-1679.036008,2


In [34]:
batches, batch_count = batcher(ngram_eval.ngram, stopwords=stop, max_batches = 10)

batches

{('311.22|CD', '500.86|CD'): 1,
 ('Makhaya|NNP', 'Ntini|NNP'): 1,
 ('resting_place_coordinates|NNS', 'burial_place|VBP'): 1,
 ('Dechawat|NNP', 'Poomjaeng|NNP'): 1,
 ('MSC1|NN', 'MSC2|NN'): 1,
 ('Deshin|NNP', 'Shekpa|NNP'): 1,
 ('Neophron|NNP', 'percnopterus|NNP'): 1,
 ('Eugeniya|NNP', 'Pashkova|NNP'): 1,
 ('Pirke|NNP', 'De-Rabbi|NNP'): 1,
 ('ISTFEA|NNP', 'ISTSPR|NNP'): 1,
 ('tikkun|NNP', 'olam|NNP'): 1,
 ('Librarie|NNP', 'Sansaisha|NNP'): 1,
 ('Frecce|NNP', 'Tricolori|NNP'): 1,
 ('Tsvetana|NNP', 'Pironkova|NNP'): 1,
 ('Ahdut|NNP', 'HaAvoda|NNP'): 1,
 ('Virgilijus|NNP', 'Alekna|NNP'): 1,
 ('Persiwa|NNP', 'Wamena|NNP'): 1,
 ('Moheener|NNP', 'Ghoraguli|NNP'): 1,
 ('Bakwa|NNP', 'Dishi|NNP'): 1,
 ('Riograndenser|NNP', 'Hunsrückisch|NNP'): 1,
 ('Toše|NNP', 'Proeski|NNP'): 1,
 ('Hifikepunye|NNP', 'Pohamba|NNP'): 1,
 ('Polie|NNP', 'Olie|NNP'): 1,
 ('benzoe|NN', '1,4diazepin|NN'): 1,
 ('Jakko|NNP', 'Jakszyk|NNP'): 1,
 ('Gm3|NN', '∕|SYM'): 1,
 ('Clivio|NNP', 'Piccione|NNP'): 1,
 ('Mensur|NNP', '

In [35]:
# Should be -2 as all stopwords
#batches[('1|CD', 'to|IN', '1|CD')]

In [36]:
len(batches)

500000

In [37]:
# Should be able to add batch information using df.map() but am encountering errors apparently relating
#  to indexing - workaround (though slower).

ngb_cols = ["ngram", "batch"]
rows = []

for ng in ngram_eval['ngram']:
    rows.append({"ngram" : ng,
                "batch" : batches[ng]})
    
ng_batch = pd.DataFrame(rows, columns = ngb_cols)

ng_batch

,ngram,batch
0,"(311.22|CD, 500.86|CD)",1
1,"(Makhaya|NNP, Ntini|NNP)",1
2,"(resting_place_coordinates|NNS, burial_place|VBP)",1
3,"(Dechawat|NNP, Poomjaeng|NNP)",1
4,"(MSC1|NN, MSC2|NN)",1
...,...,...
499995,"(US|NNP, Steel|NNP)",3
499996,"(Harry|NNP, Wilson|NNP)",3
499997,"(a|DT, solo|JJ, piano|NN)",2
499998,"(1962|CD, album|NN)",6


In [38]:
ngram_eval = ngram_eval.merge(ng_batch, on='ngram')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(311.22|CD, 500.86|CD)",20,-667.084219,2,1
1,"(Makhaya|NNP, Ntini|NNP)",20,-667.084219,2,1
2,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-667.084219,2,1
3,"(Dechawat|NNP, Poomjaeng|NNP)",20,-667.084219,2,1
4,"(MSC1|NN, MSC2|NN)",20,-667.084219,2,1
...,...,...,...,...,...
499995,"(US|NNP, Steel|NNP)",32,-1679.033919,2,3
499996,"(Harry|NNP, Wilson|NNP)",32,-1679.034986,2,3
499997,"(a|DT, solo|JJ, piano|NN)",23,-1679.035464,3,2
499998,"(1962|CD, album|NN)",30,-1679.036008,2,6


In [39]:
ngram_eval.to_pickle(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/ngram_eval.pkl')

In [40]:
ngram_eval.batch.value_counts()

 1     170225
 2     129780
 3      77191
 4      53152
 5      30841
 6      17798
 7       9470
 8       5725
 9       2410
-1       1723
 10      1561
-2        124
Name: batch, dtype: int64